<a href="https://colab.research.google.com/github/harakenken22/diveintocode-ml/blob/master/week9_scratch_ensemble.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Sprint8: アンサンブル学習
----

In [15]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


アンサンブル
https://www.loom.com/share/13abc0b6b3ae4258a15527dbb2bd5924
https://www.loom.com/share/3a8639fadbaf4e59a191c8d28f8d2867
https://www.loom.com/share/6215e2af936d4d09be2fd741a1a6fc54
https://www.loom.com/share/7ff037e5eb344b70bbc250578334cd67

In [34]:
import pandas as pd
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score
import numpy as np
from sklearn.metrics import confusion_matrix
from pandas.plotting import scatter_matrix
import itertools
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import mean_squared_error
from sklearn import metrics
import scipy.stats as stats
from sklearn.tree import DecisionTreeRegressor
from sklearn.linear_model import ElasticNet
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import lightgbm as lgb
from sklearn.linear_model import LinearRegression
from sklearn import svm
from sklearn.neighbors import KNeighborsRegressor
%matplotlib inline

住宅価格の予想からデータを引っ張って来る
---
参考URL:https://qiita.com/Moby-Dick/items/e2f1efd923fddd72b17d


In [35]:
df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/diveintocode-ml/train_houseprice.csv')

>目的変数をSalePrice, 説明変数をGrLivAreaとYearBuilt

In [36]:
X = df.loc[:, ['GrLivArea', 'YearBuilt']]
y = df.loc[:,['SalePrice']]
print(X.shape)
print(y.shape)

(1460, 2)
(1460, 1)


In [37]:
X.describe()

,GrLivArea,YearBuilt
count,1460.000000,1460.000000
mean,1515.463699,1971.267808
std,525.480383,30.202904
min,334.000000,1872.000000
25%,1129.500000,1954.000000
50%,1464.000000,1973.000000
75%,1776.750000,2000.000000
max,5642.000000,2010.000000


In [38]:
y.describe()

,SalePrice
count,1460.000000
mean,180921.195890
std,79442.502883
min,34900.000000
25%,129975.000000
50%,163000.000000
75%,214000.000000
max,755000.000000


【問題1】ブレンディングのスクラッチ実装
----
>ブレンディング をスクラッチ実装し、単一モデルより精度があがる例を 最低3つ 示してください。精度があがるとは、検証用データに対する平均二乗誤差（MSE）が小さくなることを指します。

In [43]:
clf1 = lgb.LGBMRegressor()
clf2 = DecisionTreeRegressor(max_depth=2)
clf3 = ElasticNet()
clf4 = svm.SVC()
clf5 = LinearRegression()

In [22]:
num_test = 0.20
sc = StandardScaler()
X = np.array(X)
y = np.array(y)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=num_test, random_state=23)
sc.fit(X_train)
X_train=sc.transform(X_train)

In [63]:
clf1.fit(X_train,y_train) #LGBM
y_pred1 = clf1.predict(X_test)
print(np.log(mean_squared_error(y_test, y_pred1)))
print(y_pred1.shape)
y_pred1 = y_pred1.reshape(365,1)

21.405747333685948
(365,)


/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [64]:
clf2.fit(X_train,y_train) #決定木
y_pred2 = clf2.predict(X_test)
print(np.log(mean_squared_error(y_test, y_pred2)))
print(y_pred2.shape)
y_pred2 = y_pred2.reshape(365,1)

21.945480066854184
(365,)


In [65]:
clf3.fit(X_train,y_train) #ElasticNet
y_pred3 = clf3.predict(X_test)
print(np.log(mean_squared_error(y_test, y_pred3)))
print(y_pred3.shape)
y_pred3 = y_pred3.reshape(365,1)

21.726074338451916
(365,)


In [66]:
clf4.fit(X_train,y_train) #SVM
y_pred4 = clf4.predict(X_test)
print(np.log(mean_squared_error(y_test, y_pred4)))
# print(y_pred4.shape)
y_pred4 = y_pred4.reshape(365,1)

/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


22.33451564027054


In [67]:
clf5.fit(X_train,y_train) #Liner regression
y_pred5 = clf5.predict(X_test)
print(np.log(mean_squared_error(y_test, y_pred5)))
print(y_pred5.shape)

21.726067456909853
(365, 1)


In [68]:
#シェイプの方が違うというエラーが出たのでy_testの型を確認

y_test.shape

(365, 1)

In [69]:
#LGBMと決定木と回帰分析のブレンディングを実施
y_pred_all = (y_pred1 + y_pred2 + y_pred5)/3 
print(y_pred_all.shape)
np.log(mean_squared_error(y_test, y_pred_all))

(365, 1)


21.510360090572714

In [70]:
#LGBMと決定木とSVMのブレンディングを実施
y_pred_all = (y_pred1 + y_pred2 + y_pred4)/3 
print(y_pred_all.shape)
np.log(mean_squared_error(y_test, y_pred_all))

(365, 1)


21.58595128702464

【問題2】バギングのスクラッチ実装
----
バギング をスクラッチ実装し、単一モデルより精度があがる例を 最低1つ 示してください。

回帰問題なのに分類問題を行っていた例
----

In [27]:
from sklearn.ensemble import AdaBoostRegressor
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import BaggingRegressor
X_train, X_test, y_train, y_test = train_test_split(
    X, y, random_state=0)

models = {
    'not bagging': DecisionTreeClassifier(random_state=0),
    'bagging': BaggingClassifier(DecisionTreeClassifier(random_state=0), n_estimators=100, random_state=0) 
}

scores = {}
for model_name, model in models.items():
    model.fit(X_train, y_train)
    scores[(model_name, 'train_score')] = model.score(X_train, y_train)
    scores[(model_name, 'test_score')] = model.score(X_test, y_test)

pd.Series(scores).unstack()

/usr/local/lib/python3.7/dist-packages/sklearn/ensemble/_bagging.py:645: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


,test_score,train_score
bagging,0.008219,0.957078
not bagging,0.005479,0.957078


回帰を持ちてバギングを行う
---

In [71]:
from sklearn.ensemble import AdaBoostRegressor
from sklearn.ensemble import BaggingRegressor
from sklearn.ensemble import BaggingRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
X_train, X_test, y_train, y_test = train_test_split(
    X, y, random_state=0)

models = {
    'not bagging': DecisionTreeRegressor(random_state=0),
    'bagging': BaggingRegressor(DecisionTreeRegressor(random_state=0), n_estimators=100, random_state=0) 
}

scores = {}
for model_name, model in models.items():
    model.fit(X_train, y_train)
    scores[(model_name, 'train_score')] = model.score(X_train, y_train)
    scores[(model_name, 'test_score')] = model.score(X_test, y_test)

pd.Series(scores).unstack()

/usr/local/lib/python3.7/dist-packages/sklearn/ensemble/_bagging.py:399: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return column_or_1d(y, warn=True)


,test_score,train_score
bagging,0.712119,0.950321
not bagging,0.561431,0.996900


In [77]:
from sklearn.ensemble import AdaBoostRegressor
from sklearn.ensemble import BaggingRegressor
from sklearn.ensemble import BaggingRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
X_train, X_test, y_train, y_test = train_test_split(
    X, y, random_state=0)

models = {
    'not bagging': LinearRegression(),
    'bagging': BaggingRegressor(LinearRegression(), n_estimators=100, random_state=0) 
}

scores = {}
for model_name, model in models.items():
    model.fit(X_train, y_train)
    scores[(model_name, 'train_score')] = model.score(X_train, y_train)
    scores[(model_name, 'test_score')] = model.score(X_test, y_test)

pd.Series(scores).unstack()

/usr/local/lib/python3.7/dist-packages/sklearn/ensemble/_bagging.py:399: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return column_or_1d(y, warn=True)


,test_score,train_score
bagging,0.586008,0.676913
not bagging,0.587104,0.676966


回帰を用いてバギングをを行いテスト精度を向上させた。
ランダムフォレストを用いた回帰分析の精度が向上したが、回帰分析は精度が向上しなかった。

【問題3】スタッキングのスクラッチ実装
---
スタッキング をスクラッチ実装し、単一モデルより精度があがる例を 最低1つ 示してください。

In [28]:
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsRegressor
X_train_stack, X_test_stack, y_train_stack, y_test_stack = train_test_split(
    X, y)

# train base model
base_model_1 = LinearRegression()
base_model_2 = lgb.LGBMRegressor()
base_model_3 = KNeighborsRegressor()

base_model_1.fit(X_train_stack, y_train_stack)
base_model_2.fit(X_train_stack, y_train_stack)
base_model_3.fit(X_train_stack, y_train_stack)

# base predicts
base_pred_1 = base_model_1.predict(X_test_stack)
base_pred_2 = base_model_2.predict(X_test_stack)
base_pred_3 = base_model_3.predict(X_test_stack)

/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


メタモデルの学習
----
ベースモデルの出力を使ってメタモデルの学習を行います。 そんなに難しいことはしておらず、「ベースモデルの出力を連結する」「それを入力として学習させる」だけです。

np.column_stack
---
np.column_stack((行列1, 行列2)) : １次行列の組み合わせなら２次元行列を作り、多次元の組みならhstackと同じ振る舞いをする
>https://qiita.com/gragragrao/items/1881accf27bbedc5fa84

In [29]:
# stack base predicts for training meta model
stacked_predictions = np.column_stack((base_pred_1, base_pred_2, base_pred_3))

In [30]:
# train meta model 
meta_model = LinearRegression()
meta_model.fit(stacked_predictions, y_test_stack)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [31]:
# test predicts for final result 
valid_pred_1 = base_model_1.predict(X_test_stack)
valid_pred_2 = base_model_2.predict(X_test_stack)
valid_pred_3 = base_model_3.predict(X_test_stack)
stacked_valid_predictions = np.column_stack((valid_pred_1, valid_pred_2, valid_pred_3))
meta_valid_pred = meta_model.predict(stacked_valid_predictions)

print ("mean squared error of model 1: {:.4f}".format(mean_squared_error(y_test_stack, valid_pred_1)))
print ("mean squared error of model 2: {:.4f}".format(mean_squared_error(y_test_stack, valid_pred_2)))
print ("mean squared error of model 3: {:.4f}".format(mean_squared_error(y_test_stack, valid_pred_3)))
print ("mean squared error of meta model: {:.4f}".format(mean_squared_error(y_test_stack, meta_valid_pred)))

mean squared error of model 1: 3027670828.9539
mean squared error of model 2: 2336898256.9506
mean squared error of model 3: 3346587019.7211
mean squared error of meta model: 2024197114.8408
